#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split

#### Challenge 1 : Check the current working directory status

In [2]:
os.getcwd()

'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\notebook'

In [3]:
#### As it shows the wrong directory
### Challenge 2: set the working directory to correct directory

os.chdir('e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset')
os.getcwd()

'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset'

#### Extract the data from XML Annotations and image files

In [4]:
#### In our dataset we have images folder and Annotations folder
## but we want both in one common folder so for that
### Challenge 3: copy both files in one common folder

## Now, we copied both files in one common folder

import xml.etree.ElementTree as xet
from glob import glob

#### Create XML path

In [7]:
xml_path = os.path.join(os.getcwd(), 'common', '*xml')
path = glob(xml_path)

In [8]:
path

['e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars0.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars1.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars10.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars100.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars101.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars102.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars103.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Recognition_Project\\Dataset\\common\\Cars104.xml',
 'e:\\AnacondaWorkspace\\resume\\ineuron\\Automatic_Number_Plate_Reco